# Import

Erstmal werde alle Funktionen importiert

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Tinder für Streber

## Importieren der Daten

Das sind die Daten der Unis:

In [2]:
file_path = 'DatenStudymatch.csv'

In [3]:
df = pd.read_csv(file_path)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   semester  114 non-null    int64  
 1   modul     114 non-null    object 
 2   modul_id  106 non-null    float64
 3   Credits   0 non-null      float64
 4   SWS       84 non-null     object 
 5   Uni       114 non-null    object 
 6   uni_id    114 non-null    int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 6.4+ KB


In [5]:
df.head()

,semester,modul,modul_id,Credits,SWS,Uni,uni_id
0,1,bgb_at,1.0,NaN,4,Bayreuth,3
1,1,Staatsorganisationsrecht,101.0,NaN,4,Bayreuth,3
2,1,strafrecht_1,201.0,NaN,4,Bayreuth,3
3,1,Bausteine des Rechts,301.0,NaN,2,Bayreuth,3
4,1,BWL Einführung,302.0,NaN,2,Bayreuth,3


Das sind die Daten für die Studenten:

In [6]:
file_path2 = 'Studenten.csv'

In [7]:
df2 = pd.read_csv(file_path2)

In [8]:
df2.head()

,Name,Semester,Uni
0,Lisa Müller,1,Bayreuth
1,Markus Meier,1,Münster
2,Perter Bäcker,1,Heidelberg
3,Hans Wurst,1,Bayreuth
4,Lena Gärtner,1,Münster


## Bereinigen der Daten

Wir haben uns überlegt, dass man die Credits, SWS und die Uni Id erstmal gar nicht braucht

In [9]:
data = df.drop(['Credits','SWS','uni_id'], axis=1)

Wir haben beschlossen, dass es nicht sehr viele fehlende Werte gibt und die Zeilen einfach gelöscht.

In [10]:
data = data.dropna()

Da bei den Studenten Daten Semester groß geschrieben wird, musste das bei den Uni Daten auch so sein.

In [11]:
data.rename(columns={'semester':'Semester'}, inplace=True)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 0 to 112
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Semester  106 non-null    int64  
 1   modul     106 non-null    object 
 2   modul_id  106 non-null    float64
 3   Uni       106 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 4.1+ KB


Die Daten der Studenten müssen nicht bereinigt werden. (Es muss noch Semester einheitlich geschrieben werden.)

## Erste Versuche der Datenverarbeitung

Hier haben wir uns erst überlegt die Daten in drei Dataframes, für jede Uni einen, aufzuteilen. Wri haben aber festgestellt, dass das gar nicht nötig ist.

In [13]:
#for uni in data['Uni']:
#    data['Uni'] == 'Bayreuth'
    

Wir haben uns folgende Schritte überlegt, um erstmal für ein konkretes Modul herauszufinden, an welcher Uni das in welchem Semester angeboten wird:
- Modul eingeben
- Modul wird ID zugeordnet 
- ID wird in den Daten gesucht 
- DataFrame wird gefiltert - Unis + Semester etc werden andezeigt (mit gleicher Modul_Id) 


Hier haben wir erstmal die Daten nach dem Modul Allgemeines Schuldrecht gefiltert und dessen Modul ID herausgefunden (2). <br>
Problem: Wir haben als Output einen DataFrame, wollen aber eigentlich nur die Zahl.

In [14]:
modul = data[data['modul']=='strafrecht_1']
modul.head()
#modul_id = modul['modul_id']
#print(modul_id)

,Semester,modul,modul_id,Uni
2,1,strafrecht_1,201.0,Bayreuth


Wir haben dann überlegt, ob man das durch eie Schleife lösen könnte, sind da aber auch nicht wirklich weitergekommen.

In [15]:
for modul_id in modul['modul_id']: 
    data[data['modul_id']==modul_id]
    #print(data)

Hier sind wir dann nicht weitergekommen, weil wir nicht genau wussten wo das Problem liegt. Es könnte an der Problematik von Oben (Dataframe statt Integer) oder daran liegen, dasss wir 2,0 statt 2 haben.

In [16]:
data[data['modul_id']==modul_id]

,Semester,modul,modul_id,Uni
2,1,strafrecht_1,201.0,Bayreuth
38,1,Grundkurs Strafrecht I,201.0,Heidelberg
86,1,Strafrecht 1,201.0,Münster


### Überlegungen 08.03

- Daten zusammenfügen mit .join, wie in Tabellenblatt8 zu sehen
- Das wird dann eine sehr große Tabelle mit den Personen, bei denen anhand der Uni und dem Semester Zeilen hinzugefügt werden
- Dann muss man die Daten so umwandeln, dass da nur noch die bereits gemachten Module stehen (Da das Matching über das Semester läuft, müssen wir uns noch überlegen, wie wir die vorherigen Semester auch hinzufügen - sowas wie 1 bis x(=mein Semester)
- In der Tabelle sind dann alle Module (bzw. die IDs), die die Person bis dahin schon gemacht hat drin.
- Durch die Anzahl der Module ist dann auch klar welches Semester und welche Uni gemeint ist, da die Module davon abhängig sind, wir müssen also das beim Matchen nicht beachten.
-  
- Dann wendet man eine Methode darauf an, bei der man quasi eine bestimmte Person (nicht in dem Datensatz) mit den bestehenden Personen matcht, indem geguckt wird wieviele Module sich überschneiden.
-  
- Wir matchen jetzt welche Person insgesamt am besten passt, also zum generell gemeinsam lernen und nicht nach einem bestimmten Fach!

## Daten zusammenfügen

Wir haben uns überlegt, mit .join die Dataframes zusammenzufügen und dann so umzuformen, dass für jeden Studenten alle bereits bestandenen Module zu sehen sind.

In [17]:
data_joined = data.join(df2.set_index(['Semester','Uni']), on=['Semester','Uni'])

In [18]:
data_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134 entries, 0 to 112
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Semester  134 non-null    int64  
 1   modul     134 non-null    object 
 2   modul_id  134 non-null    float64
 3   Uni       134 non-null    object 
 4   Name      45 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 6.3+ KB


Wir haben jetzt die verknüpften Daten. Bei Name gibt es viele fehlende Werte, weil alle im ersten Semester sind und somit gibt es keine Werte für die anderen Semester. Überlegung: Man könnte für jeden Studenten, der schon ein weiteres Semester gemacht hat eine weitere Zeile anlegen und da das Semester ändern. Im ersten Semester hat man also nur eine Zeile im fünften dann Fünf. (Also in der Tabelle mit den Daten der Studenten) Im Notebook oder händisch?


Die Kurse, die keiner belegt haben, sollen nicht NaN sein, sondern da soll 'Niemand' stehen:

In [19]:
data_joined = data_joined.fillna('Niemand')

In [20]:
data_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134 entries, 0 to 112
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Semester  134 non-null    int64  
 1   modul     134 non-null    object 
 2   modul_id  134 non-null    float64
 3   Uni       134 non-null    object 
 4   Name      134 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 6.3+ KB


## Daten modellieren

In [24]:
newframe = data_joined.loc[:,['Name','modul_id']]
newframe.head()

,Name,modul_id
0,Lisa Müller,1.0
0,Hans Wurst,1.0
0,Maraike Metzger,1.0
1,Lisa Müller,101.0
1,Hans Wurst,101.0


In [31]:
#newframe.set_index('modul_id')

In [32]:
#newframe.columns(newframe['modul_id'])